In [ ]:
import numpy as np
import pickle as pkl

from multiprocessing import Pool
nproc = 10

from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator

import mayavi.mlab as mlab

from athena_read import athdf

dpi = 300
save = False

In [ ]:
adiab_idx = 1.1
mach_no = 5.

if mach_no == 5.:
    isosurf_level = 0.015
    # color scale for bfield
    logbmin = np.log(10**-4)
    logbmax = np.log(0.01)
    # opacity scale for bfield
    ologbmin = logbmin
    ologbmax = np.log(0.05)
    # data sources
    dataset_pathstem = '/DATA/Dropbox/LOOTRPV/Princeton_PhD/Thesis/globAccDisk/athena/bin-mhd/M5/R4_32_dfloor1e-6_vfloorRho1e-5/'
    snapshot_path = dataset_pathstem + 'snapshots/mhdLoops_strat.out2.02308.athdf'
    # aux
    isosurf_shape = (257,256)
    cbar_fmt = '%.1f'
elif mach_no == 10.:
    isosurf_level = 0.015
    # color scale for bfield
    logbmin = np.log(10**-4)
    logbmax = np.log(0.01)
    # opacity scale for bfield
    ologbmin = np.log(10**-4)
    ologbmax = np.log(0.15)
    # data sources
    dataset_pathstem = '/DATA/Dropbox/LOOTRPV/Princeton_PhD/Thesis/globAccDisk/athena/bin-mhd/M10_noInfl/R5_4b_lowDfloor/R5_dfloor1e-6_vfloorRho1e-5/'
    snapshot_path = dataset_pathstem + 'snapshots/mhdLoops_strat.out2.00908.athdf'
    # aux
    isosurf_shape = (513,512)
    cbar_fmt = '%.2f'

In [ ]:
with open(dataset_pathstem + 'steady_rho3D.pkl', 'rb') as f:
    data3D = pkl.load(f)
    
with open(dataset_pathstem + 'steady_csound3D.pkl', 'rb') as f:
    data_csound = pkl.load(f)
with open(dataset_pathstem + 'steady_bfield3D.pkl', 'rb') as f:
    data_bfield = pkl.load(f)
    
data = athdf(snapshot_path)

use_snapshot_csound = True
use_snapshot_bfield = True

# override to just show a snapshot
data3D.val = np.transpose(data['rho'], axes=[2,1,0])

In [ ]:
if False:
    import matplotlib.pyplot as plt

    r, theta, phi = np.meshgrid(data3D.r, data3D.theta, data3D.phi, indexing='ij')
    print(phi.shape)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='polar')
    plt.tricontourf(phi[:,256,:].flatten(), r[:,256,:].flatten(), np.log10(data3D.val[:,256,:]).flatten())
    plt.colorbar()
    plt.show()
    plt.close()

In [ ]:
def isborder (r_idx, phi_idx, data3D, level):
    pencil = data3D.val[min(len(data3D.r)-1,max(0,r_idx)), :, phi_idx % len(data3D.phi)]
    higher = np.where(pencil > level)[0]
    return (len(higher) > 0)

# for each phi,r (spherical) pencil, return height R,phi,z (cylindrical) of the highest and lowest density isosurface
def isopoints (level, phi_idx, r_idx, data3D):
    pencil = data3D.val[r_idx, :, phi_idx % len(data3D.phi)]
    higher = np.where(pencil > level)[0]
    phi = data3D.phi[phi_idx % len(data3D.phi)]
    r = data3D.r[r_idx]
    # sew the borders, ignore the rest
    if len(higher) == 0:
        if isborder(r_idx-1, phi_idx, data3D, level) \
        or isborder(r_idx+1, phi_idx, data3D, level) \
        or isborder(r_idx, phi_idx-1, data3D, level) \
        or isborder(r_idx, phi_idx+1, data3D, level) \
        or isborder(r_idx-1, phi_idx-1, data3D, level) \
        or isborder(r_idx+1, phi_idx+1, data3D, level) \
        or isborder(r_idx-1, phi_idx-1, data3D, level) \
        or isborder(r_idx+1, phi_idx+1, data3D, level):
            return [r, phi, 0.5*np.pi, 0.5*np.pi]
        else:
            return [r, phi, np.nan, np.nan]
    theta_idx_min, theta_idx_max = min(higher), max(higher)
    theta_min, theta_max = data3D.theta[theta_idx_min], data3D.theta[theta_idx_max]
    return [r, phi, theta_min, theta_max]

def isosurface (level, data3D):
    idxs_phi, idxs_r = np.meshgrid(range(len(data3D.phi)+1), range(len(data3D.r)), indexing='ij')
    idxs_phi = idxs_phi.flatten()
    idxs_r = idxs_r.flatten()
    points = map(lambda phi_idx, r_idx : isopoints(level, phi_idx, r_idx, data3D), idxs_phi, idxs_r)
    points = np.array(list(points))
    points = points[points[:,0] != None]
    return np.array(points).transpose().astype(np.float)

In [ ]:
surf = isosurface(isosurf_level, data3D)

In [ ]:
xx = {}; yy = {}; zz = {}; cc = {}
for i in [2,3]:

    r = surf[0]
    phi = surf[1]
    theta = surf[i]

    r_cyl = r * np.sin(theta)
    z_top = r * np.cos(theta)
    
    def z_fun (pt):
        return griddata((r_cyl, phi), z_top, xi=pt)

    #cs_fun = RegularGridInterpolator((data['x3v'], data['x2v'], data['x1v']), np.sqrt(adiab_idx * data['press']/data['rho']), bounds_error=False)
    if use_snapshot_csound:
        temp_fun = RegularGridInterpolator((data['x3v'], data['x2v'], data['x1v']), data['press']/data['rho'], bounds_error=False)
    else:
        temp_fun = RegularGridInterpolator((data_csound.r, data_csound.theta, data_csound.phi), data_csound.val, bounds_error=False)
    
    rr = r_cyl.reshape(isosurf_shape)
    pp = phi.reshape(isosurf_shape)
    tt = theta.reshape(isosurf_shape)
    zz[i] = z_top.reshape(isosurf_shape)

    xx[i] = rr *np.cos(pp)
    yy[i] = rr*np.sin(pp)

    if use_snapshot_csound:
        points = np.array([pp.flatten(), tt.flatten(), rr.flatten()]).transpose()
    else:
        points = np.array([rr.flatten(), tt.flatten(), pp.flatten()]).transpose()

    with Pool(nproc) as pool:
        #cc[i] = np.array(pool.map(cs_fun, points)).reshape(rr.shape)
        cc[i] = np.array(pool.map(temp_fun, points)).reshape(rr.shape)
        

In [ ]:
if use_snapshot_bfield:
    pp = data['x3v']
else:
    pp = data_bfield.phi
    
# stitch in phi direction
dphi = pp[1] - pp[0]
phi = np.array([pp[0]-dphi,] + list(pp) + [pp[-1]+dphi,])
if use_snapshot_bfield:
    bfield = np.sqrt(data['Bcc1']**2+data['Bcc2']**2+data['Bcc3']**2)
else:
    bfield = np.transpose(data_bfield.val, axes=[2,1,0])

bfield = np.array([bfield[-1,:,:],] + list(bfield) + [bfield[0,:,:],])

b_fun = RegularGridInterpolator((phi, data['x2v'], data['x1v']), bfield, bounds_error=False)

x = np.linspace(-0.6, 0.6, 128)
y = np.linspace(-0.6, 0.6, 128)
z = np.linspace(-0.6, 0.6, 128)

xg, yg, zg = np.meshgrid(x, y, z, indexing='ij')

r = np.sqrt(xg**2+yg**2+zg**2)
r_cyl = np.sqrt(xg**2+yg**2)
theta = np.arccos(zg/r)
phi = np.where(yg > 0, np.arccos(xg/r_cyl), 2.*np.pi - np.arccos(xg/r_cyl))

points = np.array([phi.flatten(), theta.flatten(), r.flatten()]).transpose()

with Pool(nproc) as pool:
    b = np.array(pool.map(b_fun, points)).reshape(xg.shape)

In [ ]:
# initialize mayavi
fig = mlab.figure(bgcolor=(1,1,1), fgcolor=(0,0,0), size=(int(3.5*dpi), int(3.5*dpi)))

In [ ]:
if False:
    vmin=np.nanmin(cc[i]); vmax=np.nanmax(cc[i])
else:
    vmin = 0.
    if mach_no == 5.:
        vmax = 0.6
    elif mach_no == 10.:
        vmax = 0.15

# plot the isocontours
for i in [2,3]:
    obj1 = mlab.mesh(xx[i],yy[i],zz[i], scalars=cc[i], colormap='hot', vmin=vmin, vmax=vmax)

# add the colorbar
cbar = mlab.colorbar(object=obj1, title='Temperature', orientation='horizontal', label_fmt=cbar_fmt, nb_labels=4)

# add the x axis
_ = mlab.outline(extent=[-0.35,0.6, -0.35,0.35, -0.1,0.1])
maxes = mlab.axes(color=(0,0,0), line_width=2, nb_labels=2, x_axis_visibility=True, y_axis_visibility=True, z_axis_visibility=True, xlabel='', ylabel='', zlabel='')
# add later: label_format='%.2f', corner_offset=0.05)

In [ ]:
# patch to fix the mayavi volume color bug, doesn't seem to help...
logb = np.log(b)
if False:
    # scale b to 0-255 to avoid mayavi volume color bug
    logb = np.where(logb >= logbmin, logb - logbmin, 0.)
    logb = np.where(logb <= (logbmax-logbmin), logb/(logbmax-logbmin), 1.)
    logb *= 255

s = mlab.pipeline.scalar_field(xg, yg, zg, logb)
obj2 = mlab.pipeline.volume(s, vmin=logbmin, vmax=logbmax)

# Change the colormap
if False:
    values = np.linspace(0., 1., 256)
    from tvtk.util import ctf
    from matplotlib.pyplot import cm
    c = ctf.save_ctfs(obj2._volume_property)
    c['rgb']=cm.get_cmap('jet')(values.copy())
    ctf.load_ctfs(c, obj2._volume_property)
    obj2.update_ctf = True

# Changing the otf:
if True:
    from tvtk.util.ctf import PiecewiseFunction
    otf = PiecewiseFunction()
    otf.add_point(ologbmin, 0.)
    otf.add_point(ologbmax, 1.)
    obj2._otf = otf
    obj2._volume_property.set_scalar_opacity(otf)

obj2.update_ctf = True

# add the colorbar
if False:
    mlab.colorbar(object=obj2, title='B [sim.u.]', orientation='horizontal')
    
# add the H/R angle
if False:
    x = np.array([0.,0.])
    y = np.array([0.,0.35])
    z = y / mach_no
    _ = mlab.plot3d(x,y,z, line_width=0.1, tube_radius=0.01)

In [ ]:
# set the camera orientation
mlab.view(azimuth=0., elevation=55.) #300
mlab.move(up=-0.15)

In [ ]:
mlab.draw()
if save:
    mlab.savefig('3Dplot.png', figure=fig)
else:
    mlab.show()

In [ ]:
mlab.close()